In [15]:
import torch
import torch.nn as nn
from torchsummary import summary
import numpy as np

In [16]:
file_path = "/Users/siddharth/Downloads/modelnet10.npz"
data = np.load(file_path, allow_pickle=True)
train_voxel = data["train_voxel"] # Training 3D voxel samples
test_voxel = data["test_voxel"] # Test 3D voxel samples
train_labels = data["train_labels"] # Training labels (integers from 0 to 9)
test_labels = data["test_labels"] # Test labels (integers from 0 to 9)
class_map = data["class_map"] # Dictionary mapping the labels to their class names.

In [24]:

# Check the shape of your data
data_shape = train_voxel.shape if isinstance(train_voxel, np.ndarray) else train_voxel.size()
num_channels = data_shape[1] if len(data_shape) >= 2 else 1  # Assumes channels are in the second dimension

print("Number of Channels in Input Data:", num_channels)


Number of Channels in Input Data: 64


In [22]:
import torch
import torch.nn as nn

class Generator(torch.nn.Module):
    def __init__(self, in_channels=512, out_dim=64, out_channels=1, noise_dim=200, activation="sigmoid"):
        super(Generator, self).__init__()
        self.in_channels = in_channels
        self.out_dim = out_dim
        self.in_dim = int(out_dim / 16)
        conv1_out_channels = int(self.in_channels / 2.0)
        conv2_out_channels = int(conv1_out_channels / 2)
        conv3_out_channels = int(conv2_out_channels / 2)

        self.linear = torch.nn.Linear(noise_dim, in_channels * self.in_dim * self.in_dim * self.in_dim)

        self.conv1 = nn.Sequential(
            nn.ConvTranspose3d(
                in_channels=in_channels, out_channels=conv1_out_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm3d(conv1_out_channels),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.ConvTranspose3d(
                in_channels=conv1_out_channels, out_channels=conv2_out_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm3d(conv2_out_channels),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.ConvTranspose3d(
                in_channels=conv2_out_channels, out_channels=conv3_out_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm3d(conv3_out_channels),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.ConvTranspose3d(
                in_channels=conv3_out_channels, out_channels=out_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )
        if activation == "sigmoid":
            self.out = torch.nn.Sigmoid()
        else:
            self.out = torch.nn.Tanh()

    def project(self, x):
        """
        projects and reshapes latent vector to starting volume
        :param x: latent vector
        :return: starting volume
        """
        return x.view(-1, self.in_channels, self.in_dim, self.in_dim, self.in_dim)

    def forward(self, x):
        x = self.linear(x)
        x = self.project(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return self.out(x)


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the Discriminator
class Discriminator(torch.nn.Module):
    def __init__(self, in_channels=1, dim=64, out_conv_channels=512):
        super(Discriminator, self).__init__()
        conv1_channels = int(out_conv_channels / 8)
        conv2_channels = int(out_conv_channels / 4)
        conv3_channels = int(out_conv_channels / 2)
        self.out_conv_channels = out_conv_channels
        self.out_dim = int(dim / 16)

        self.conv1 = nn.Sequential(
            nn.Conv3d(
                in_channels=in_channels, out_channels=conv1_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm3d(conv1_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv3d(
                in_channels=conv1_channels, out_channels=conv2_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm3d(conv2_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv3d(
                in_channels=conv2_channels, out_channels=conv3_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm3d(conv3_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv3d(
                in_channels=conv3_channels, out_channels=out_conv_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm3d(out_conv_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.out = nn.Sequential(
            nn.Linear(out_conv_channels * self.out_dim * self.out_dim * self.out_dim, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(-1, self.out_conv_channels * self.out_dim * self.out_dim * self.out_dim)
        x = self.out(x)
        return x




In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define a function to create random noise
def generate_noise(batch_size, noise_dim):
    return torch.rand(batch_size, noise_dim)

# Create the Generator and Discriminator instances
generator = Generator(in_channels=512, out_dim=64, out_channels=1, noise_dim=200)
discriminator = Discriminator()

# Define the loss and optimizers
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer_generator = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=0.0002)

# Training parameters
num_epochs = 3
batch_size = 64  # You can adjust this within the range of 10 to 100

# Assuming you have loaded your dataset into train_voxel, convert it to PyTorch tensors
train_voxel = torch.tensor(train_voxel, dtype=torch.float32)

# Create a DataLoader for batch training
train_dataset = TensorDataset(train_voxel)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for batch_data in train_loader:
        real_data = batch_data[0]  # Assuming your data is structured as a single tensor
        batch_size = real_data.size(0)
        
        # Train the Discriminator
        optimizer_discriminator.zero_grad()
        noise = generate_noise(batch_size, noise_dim)
        fake_data = generator(noise)
        
        # Real data target
        target_real = Variable(torch.ones(batch_size, 1))
        loss_real = criterion(discriminator(real_data), target_real)
        
        # Fake data target
        target_fake = Variable(torch.zeros(batch_size, 1))
        loss_fake = criterion(discriminator(fake_data.detach()), target_fake)
        
        loss_discriminator = loss_real + loss_fake
        loss_discriminator.backward()
        optimizer_discriminator.step()
        
        # Train the Generator
        optimizer_generator.zero_grad()
        noise = generate_noise(batch_size, noise_dim)
        fake_data = generator(noise)
        target_real = Variable(torch.ones(batch_size, 1))
        loss_generator = criterion(discriminator(fake_data), target_real)
        loss_generator.backward()
        optimizer_generator.step()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"Discriminator Loss: {loss_discriminator.item()}, Generator Loss: {loss_generator.item()}")

# Save your generator and discriminator models if needed
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")


/var/folders/fr/p253sg2536gclcy0s7m_d7940000gn/T/ipykernel_41109/854940844.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_voxel = torch.tensor(train_voxel, dtype=torch.float32)


RuntimeError: Given groups=1, weight of size [64, 1, 4, 4, 4], expected input[1, 64, 64, 64, 64] to have 1 channels, but got 64 channels instead